<a href="https://colab.research.google.com/github/louis-not/Notogo-ML/blob/main/model_v2.2/Hyperparameter_Tuning_V2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RUN

## Import Module

### For Optimization

In [ ]:
!pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=ee7d80a339d29fe0e4316cfc0e8fd471da3009e2b499a4d878e53e1eb8049d4a
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [ ]:
import numpy as np
from bayes_opt import BayesianOptimization

### For Model

In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     |████████████████████████████████| 85 kB 4.0 MB/s 
     |████████████████████████████████| 462 kB 44.4 MB/s 
     |████████████████████████████████| 4.2 MB 12.8 MB/s 


In [2]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import pandas as pd

import tensorflow_recommenders as tfrs

# getting data
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import get_as_dataframe, set_with_dataframe

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Bangkit/Capstone/ML/ML-GH/Notogo-ML/"model_v2.2"

In [5]:
cd /content/drive/Othercomputers/My Laptop/Bangkit/Capstone/Recommender system/Notogo-ML/model_v2.2_Deploy

/content/drive/Othercomputers/My Laptop/Bangkit/Capstone/Recommender system/Notogo-ML/model_v2.2_Deploy


In [ ]:
print(os.getcwd())

## Retrieval Model

### Preparing Dataset

In [6]:
import userFeatures
builder = tfds.builder('Userfeatures')
userFeatureDs = tfds.load('Userfeatures',split='train')

import wishEmbedding
builder = tfds.builder('Wishembedding')
wishEmbeddingDs = tfds.load('Wishembedding',split='train')

ratings = userFeatureDs.map(lambda x: {
    "location_id": x["location_id"],
    "user_id": x["user_id"],
    "add" : x["add"],
    "like" : x['like']
})
locations = wishEmbeddingDs.map(lambda x: x["location_id"])

for x in ratings.take(2).as_numpy_iterator():
  pprint.pprint(x)

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/userfeatures/1.0.0.incomplete25ZVKA/userfeatures-train.tfrecord*...:   0%|…

Dataset userfeatures downloaded and prepared to /root/tensorflow_datasets/userfeatures/1.0.0. Subsequent calls will reuse this data.


Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/wishembedding/1.0.0.incompleteHCE0MI/wishembedding-train.tfrecord*...:   0…

Dataset wishembedding downloaded and prepared to /root/tensorflow_datasets/wishembedding/1.0.0. Subsequent calls will reuse this data.
{'add': 0, 'like': 1, 'location_id': b'61', 'user_id': b'93'}
{'add': 0, 'like': 1, 'location_id': b'53', 'user_id': b'112'}


In [7]:
tf.random.set_seed(42)

NUM_DATA = ratings.__len__().numpy()

shuffled = ratings.shuffle(NUM_DATA, seed=42, reshuffle_each_iteration=False)

trainset_size = 0.8 * NUM_DATA

train = shuffled.take(trainset_size)
test = shuffled.skip(trainset_size).take(NUM_DATA - trainset_size)

location_name = locations.batch(1000)
user_ids = ratings.batch(1000).map(lambda x: x["user_id"])

unique_location_name = np.unique(np.concatenate(list(location_name)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

cached_train = train.shuffle(NUM_DATA).batch(512).cache()
cached_test = test.batch(256).cache()

### Model
change this if needed for a different model

In [8]:
class NoToGoModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, like_weight: float,retrieval_weight: float, 
               BATCH_SIZE, EMB_DIM
               ) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = int(EMB_DIM)

    # User and movie models.
    self.location_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_location_name, mask_token=None),
      tf.keras.layers.Embedding(len(unique_location_name) + 1, embedding_dimension),
      tf.keras.layers.Dense(16, activation="relu")
    ])

    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension),
      tf.keras.layers.Dense(16, activation="relu")
    ])

    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(8, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1, activation = "sigmoid"),
    ])

    self.like_model = tf.keras.Sequential([
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(1, activation = "sigmoid"),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )

    self.like_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=locations.batch(BATCH_SIZE).map(self.location_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight
    self.like_weight = like_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model.
    location_embeddings = self.location_model(features["location_id"])
    
    return (
        user_embeddings,
        location_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings,location_embeddings], axis=1)
        ),
        self.like_model(
            tf.concat([user_embeddings,location_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("add","like")
    like = features.pop("like", "add")

    user_embeddings, location_embeddings, rating_predictions, like_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )

    like_loss = self.like_task(
        labels=like,
        predictions=like_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, location_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss + like_loss*self.like_weight)

## Bayes Optimization

In [ ]:
def black_box(lr,emb_dim, epoch):
  """black box function for optimization used for Retrieval-specialized model"""
  METRIC = 'factorized_top_k/top_1_categorical_accuracy'
  batch = 180


  # Initialize model
  model = NoToGoModel(rating_weight=0.0, like_weight = 0, retrieval_weight=1.0,
                      BATCH_SIZE=batch, EMB_DIM=emb_dim)
  model.compile(optimizer=tf.keras.optimizers.Adagrad(lr))

  # training model
  model.fit(cached_train, epochs=int(epoch), verbose=0)
  result = model.evaluate(cached_train, return_dict=True, verbose=0)

  return result[METRIC]

In [ ]:
param_bounds = {
    # 'batch' : (150, 200),
    'lr' : (0.001,0.1),
    'emb_dim' : (16, 128),
    'epoch' : (5, 25)
}

optimizer = BayesianOptimization(
    f=black_box,
    pbounds=param_bounds,
    random_state=1,
)

In [ ]:
optimizer.maximize(
    init_points=2, #How many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.
    n_iter=50, #How many steps of bayesian optimization you want to perform. The more steps the more likely to find a good maximum you are.
)

|   iter    |  target   |  emb_dim  |   epoch   |    lr     |
-------------------------------------------------------------


KeyboardInterrupt: ignored

In [ ]:
print(optimizer.max)

## Evaluate

In [9]:
# Result :
batch = 180
emb_dim = 41
epoch = 8
lr = 0.09478

In [10]:
model = NoToGoModel(rating_weight=0.0, like_weight = 0, retrieval_weight=1.0,
                    BATCH_SIZE=batch, EMB_DIM=emb_dim)
model.compile(optimizer=tf.keras.optimizers.Adagrad(lr))

# training model
model.fit(cached_train, epochs=int(epoch))
model.evaluate(cached_train, return_dict=True)

Epoch 1/8
49/49 [==============================] - 8s 78ms/step - root_mean_squared_error: 0.4986 - factorized_top_k/top_1_categorical_accuracy: 0.2840 - factorized_top_k/top_5_categorical_accuracy: 0.4426 - factorized_top_k/top_10_categorical_accuracy: 0.5146 - factorized_top_k/top_50_categorical_accuracy: 0.7016 - factorized_top_k/top_100_categorical_accuracy: 0.8564 - loss: 3071.3944 - regularization_loss: 0.0000e+00 - total_loss: 3071.3944
Epoch 2/8
49/49 [==============================] - 4s 75ms/step - root_mean_squared_error: 0.4981 - factorized_top_k/top_1_categorical_accuracy: 0.9335 - factorized_top_k/top_5_categorical_accuracy: 0.9582 - factorized_top_k/top_10_categorical_accuracy: 0.9636 - factorized_top_k/top_50_categorical_accuracy: 0.9994 - factorized_top_k/top_100_categorical_accuracy: 0.9997 - loss: 3071.2400 - regularization_loss: 0.0000e+00 - total_loss: 3071.2400
Epoch 3/8
49/49 [==============================] - 4s 77ms/step - root_mean_squared_error: 0.4980 - fact

{'factorized_top_k/top_100_categorical_accuracy': 1.0,
 'factorized_top_k/top_10_categorical_accuracy': 0.9505912065505981,
 'factorized_top_k/top_1_categorical_accuracy': 0.9138596653938293,
 'factorized_top_k/top_50_categorical_accuracy': 1.0,
 'factorized_top_k/top_5_categorical_accuracy': 0.9384015202522278,
 'loss': 124.49578094482422,
 'regularization_loss': 0,
 'root_mean_squared_error': 0.49772095680236816,
 'total_loss': 124.49578094482422}

In [11]:
model.evaluate(cached_train, return_dict=True)

49/49 [==============================] - 7s 133ms/step - root_mean_squared_error: 0.4977 - factorized_top_k/top_1_categorical_accuracy: 0.9139 - factorized_top_k/top_5_categorical_accuracy: 0.9384 - factorized_top_k/top_10_categorical_accuracy: 0.9506 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 3071.0968 - regularization_loss: 0.0000e+00 - total_loss: 3071.0968


{'factorized_top_k/top_100_categorical_accuracy': 1.0,
 'factorized_top_k/top_10_categorical_accuracy': 0.9505912065505981,
 'factorized_top_k/top_1_categorical_accuracy': 0.9138596653938293,
 'factorized_top_k/top_50_categorical_accuracy': 1.0,
 'factorized_top_k/top_5_categorical_accuracy': 0.9384015202522278,
 'loss': 124.49578094482422,
 'regularization_loss': 0,
 'root_mean_squared_error': 0.49772095680236816,
 'total_loss': 124.49578094482422}

In [12]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((locations.batch(100), locations.batch(100).map(model.location_model)))
)

# Get recommendations.
user_id = "0"
_, titles = index(tf.constant([user_id]))
print(f"Recommendations for New User : {titles}")

Recommendations for New User : [[b'39' b'135' b'55' b'28' b'64' b'90' b'81' b'6' b'122' b'50']]


## DEPLOYMENT

In [13]:
path = os.path.join(os.getcwd(), "model")

# Save the index.
tf.saved_model.save(index, path)

# Load it back; can also be done in TensorFlow Serving.
loaded = tf.saved_model.load(path)

# Pass a user id in, get top predicted movie titles back.
scores, titles = loaded(["183"])

print(f"Recommendations: {titles[0][:5]}")

INFO:tensorflow:Assets written to: /content/drive/Othercomputers/My Laptop/Bangkit/Capstone/Recommender system/Notogo-ML/model_v2.2_Deploy/model/assets


INFO:tensorflow:Assets written to: /content/drive/Othercomputers/My Laptop/Bangkit/Capstone/Recommender system/Notogo-ML/model_v2.2_Deploy/model/assets


Recommendations: [b'39' b'135' b'55' b'28' b'64']


In [ ]:
MAX_LIM = 30

dfRecommendation = []

for user_id in unique_user_ids:
  _, recommendation = index(tf.constant([user_id]))
  recommendation = recommendation[0, :MAX_LIM].numpy().tolist()
  # print(str(user_id) + ' : ' + str(recommendation))
  recommendation = [ x.decode("utf-8") for x in recommendation ]
  dfRecommendation.append(recommendation)

In [ ]:
dfRecommendation = pd.DataFrame(dfRecommendation)
dfRecommendation.head()

,0,1,2,3,4,5,6,7,8,9
0,26,30,38,39,55,28,64,90,81,6
1,26,30,62,54,38,22,101,39,55,28
2,27,71,63,53,118,66,17,91,78,22
3,39,55,28,64,90,81,6,50,47,33
4,26,30,62,38,54,22,101,39,55,28


In [ ]:
dfRecommendation.reset_index(inplace=True)

In [ ]:
dfRecommendation.rename(columns={'index':'user_id'}, inplace=True)

In [ ]:
path = os.path.join(os.getcwd(),'recommendation.csv')
print(path)

/content/drive/MyDrive/Bangkit/Capstone/ML/ML-GH/Notogo-ML/model_v2.2/recommendation.csv


In [ ]:
dfRecommendation.to_csv(path,index=False)